---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [179]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

def blight_model():
    
    # Your code here
    #data_train = pd.read_csv('readonly/train.csv',encoding = "ISO-8859-1")
    #data_test = pd.read_csv('readonly/test.csv')
    data_train = pd.read_csv('train.csv',encoding = "ISO-8859-1")
    data_test = pd.read_csv('test.csv')
    
    
    #print(data_train.columns)
    
    col_train = ['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status','compliance']
    
    col_test = ['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status']
    
    data_train_clean = data_train[col_train]
    data_test_clean = data_test[col_test]
       
    data_train_clean.drop(['agency_name','inspector_name','violator_name','violation_street_name','violation_zip_code',
                           'violation_code','mailing_address_str_name','violation_description','city','state','non_us_str_code',
                           'country','disposition','grafitti_status','ticket_issued_date','hearing_date','zip_code','mailing_address_str_number'],axis=1, inplace=True)
    data_test_clean.drop(['agency_name','inspector_name','violator_name','violation_street_name','violation_zip_code',
                           'violation_code','mailing_address_str_name','violation_description','city','state','non_us_str_code',
                           'country','disposition','grafitti_status','ticket_issued_date','hearing_date','zip_code','mailing_address_str_number'],axis=1, inplace=True)

    data_train_clean = data_train_clean.dropna()
    data_test_clean = data_test_clean.dropna()
    
    y_train = data_train_clean['compliance']
    data_train_clean.drop(['compliance'],axis=1,inplace=True)
    X_test = data_test_clean
    
    lr = LogisticRegression().fit(data_train_clean, y_train)
    lr_predicted = lr.predict_proba(X_test)
    
    prob = [p1 for p0,p1 in lr_predicted]
    #print(lr.score(data_train_clean,y_train))
    
    return pd.Series(prob,index =X_test['ticket_id'])

In [180]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


ticket_id
284932    5.374340e-02
285362    2.835473e-03
285361    7.062152e-02
285338    5.747580e-02
285346    8.084133e-02
285345    5.753918e-02
285347    9.548396e-02
285342    2.383117e-01
285530    3.231902e-03
284989    1.896324e-02
285344    9.545933e-02
285343    3.279495e-03
285340    3.281263e-03
285341    9.550603e-02
285349    8.087415e-02
285348    5.756313e-02
284991    1.896742e-02
285532    1.978506e-02
285406    1.780908e-02
285001    2.000778e-02
285006    3.279700e-03
285405    2.823998e-03
285337    1.712425e-02
285496    9.670036e-02
285497    5.830326e-02
285378    2.814414e-03
285589    1.727875e-02
285585    5.068034e-02
285501    8.136200e-02
285581    2.806269e-03
              ...     
376367    7.298470e-03
376366    4.362695e-02
376362    3.988043e-02
376363    4.748720e-02
376365    7.298478e-03
376364    4.362700e-02
376228    4.577270e-02
376265    4.119764e-02
376286    2.435383e-01
376320    4.450275e-02
376314    4.147346e-02
376327    2.737529e-01
3

In [ ]:
#Separate the data into distinct groups by similarity


In [ ]:
'''
Trees often require less preprocessing of data


Trees are easy to interpret and visualize


Trees are naturally resistant to overfitting

----------------------------------
Trees are naturally resistant to overfitting
----------

Trees often require less preprocessing of data


Trees are naturally resistant to overfitting

-----------

Trees are easy to interpret and visualize




Trees often require less preprocessing of data

'''



In [ ]:
'''
To improve generalization by reducing correlation among the trees and making the model more robust to bias.
'''

In [ ]:
'''
KNN

Support Vector Machines


Neural Networks
'''

In [ ]:
'''
For a model that won’t overfit a training set, Naive Bayes would be a better choice than a decision tree.


For predicting future sales of a clothing line, Linear regression would be a better choice than a decision tree regressor.


For a fitted model that doesn’t take up a lot of memory, KNN would be a better choice than logistic regression.
----------------

For predicting future sales of a clothing line, Linear regression would be a better choice than a decision tree regressor.


For having an audience interpret the fitted model, a support vector machine would be a better choice than a decision tree.

------

For predicting future sales of a clothing line, Linear regression would be a better choice than a decision tree regressor.


For a model that won’t overfit a training set, Naive Bayes would be a better choice than a decision tree.


For having an audience interpret the fitted model, a support vector machine would be a better choice than a decision tree.


---------


For predicting future sales of a clothing line, Linear regression would be a better choice than a decision tree regressor.


For a model that won’t overfit a training set, Naive Bayes would be a better choice than a decision tree.


'''

In [ ]:
'''
Neural Network
KNN (k=1)
Decision Tree
'''


In [ ]:
'''
0.06745
'''

In [ ]:
'''
compliance_detail - More information on why each ticket was marked compliant or non-compliant


collection_status - Flag for payments in collections


grafitti_status - Flag for graffiti violations


agency_name - Agency that issued the ticket
------

ticket_issued_date - Date and time the ticket was issued

-----

grafitti_status - Flag for graffiti violations


compliance_detail - More information on why each ticket was marked compliant or non-compliant


agency_name - Agency that issued the ticket
-------


collection_status - Flag for payments in collections




compliance_detail - More information on why each ticket was marked compliant or non-compliant


'''






In [ ]:
'''
If time is a factor, remove any data related to the event of interest that doesn’t take place prior to the event.

Remove variables that a model in production wouldn’t have access to

-----------
Remove variables that a model in production wouldn’t have access to

------

Ensure that data is preprocessed outside of any cross validation folds.


Remove variables that a model in production wouldn’t have access to


Sanity check the model with an unseen validation set



Ensure that data is preprocessed outside of any cross validation folds.


Remove variables that a model in production wouldn’t have access to


Sanity check the model with an unseen validation set

'''

In [ ]:
'''
x1	x2	output
0	0	0
0	1	0
1	0	0
1	1	1
--------

x1	x2	output
0	0	0
0	1	1
1	0	1
1	1	0

'''